## Running CFD simulations

In [1]:
from alcfd import *

## Bent Pipe System

### Defining input parameters

In [2]:
pipe_D = np.array([0.1,0.02, 0.08, 0.5]) # [m]
bend_angle = np.array([1, 1, 1, 1]) # [degrees]
rho = np.array([1.225, 997, 997, 1.225]) # [kg m^-3]
muo = np.array([1.7894e-5, 8.9e-4, 8.9e-4, 1.7894e-5]) # [Pa s]
inlet_v = np.array([0.1, 0.05, 0.01, 0.05]) # [m/s]
inlet_p = np.array([101325, 101325, 101325, 101325]) # [Pa]
inputs = [pipe_D, bend_angle, inlet_v, rho, muo, inlet_p]

### Check if flow is laminar

In [3]:
re_number = check_laminar(pipe_D, inlet_v, rho, muo)
print(re_number)

(array([ 684.58701241, 1120.2247191 ,  896.17977528, 1711.46753102]), array([], dtype=int64))


### Run CFD simulations

In [4]:
outputs = run_bent_pipe_model(*inputs, l_d_ratio=10, name='validation')
outputs

Inputs to the model are:
      D  L_D_ratio  Angle  Density  Viscosity  V_in    P_in  \
0  0.10         10      1    1.225   0.000018  0.10  101325   
1  0.02         10      1  997.000   0.000890  0.05  101325   
2  0.08         10      1  997.000   0.000890  0.01  101325   
3  0.50         10      1    1.225   0.000018  0.05  101325   

                                        outputs_file  
0  /gpfs/fs2/scratch/mgholiza/test_installable_al...  
1  /gpfs/fs2/scratch/mgholiza/test_installable_al...  
2  /gpfs/fs2/scratch/mgholiza/test_installable_al...  
3  /gpfs/fs2/scratch/mgholiza/test_installable_al...  
Model inputs are saved in:
 /gpfs/fs2/scratch/mgholiza/test_installable_alcfd/alcfd/examples/inputs_validation.txt
Results are stored in:
 /gpfs/fs2/scratch/mgholiza/test_installable_alcfd/alcfd/examples/outputs_validation.txt.


,Pipe D,Elbow angle,Mesh size,Density,Viscosity,Inlet V,Inlet P,Pipe L/D,Del P/L
0,0.10,1.0,0.005,1.225,0.000018,0.10,101325.0,10.0,0.005785
1,0.02,1.0,0.001,997.000,0.000890,0.05,101325.0,10.0,3.616446
2,0.08,1.0,0.004,997.000,0.000890,0.01,101325.0,10.0,0.045080
3,0.50,1.0,0.025,1.225,0.000018,0.05,101325.0,10.0,0.000117


### Compare with baseline

In [5]:
del_pl_hp = compute_Hagen_Poiseuille_del_PL(muo, inlet_v, pipe_D)   
print(del_pl_hp)

[5.726080e-03 3.560000e+00 4.450000e-02 1.145216e-04]


In [6]:
error  = np.mean((outputs['Del P/L'].values - del_pl_hp) / del_pl_hp *100)
print(f'Average error of the model is {error :0.2f} %.')

Average error of the model is 1.52 %.


## Backflow in expansion joint

Follow the same steps as the bent pipe systems, change the input params and use `run_expansion_model()` to run CFD simulations for this system.

In [7]:
# note the unit for inlet_D
inlet_D = np.array([200]) # [um]
expansion_angle = np.array([40]) # [degrees]
inlet_v = np.array([0.5]) # [m/s]
inputs = [inlet_D, expansion_angle, inlet_v]
# Using blood as fluid
rho = 1060 # [kg m^-3]
muo = 0.004 # [Pa s]
re_number = check_laminar(inlet_D*1e-6, inlet_v, rho, muo)
print(re_number)

(array([26.5]), array([], dtype=int64))


In [8]:
outputs = run_expansion_model(*inputs, name='par_test', debug=True, run_parallel=True)
outputs

Inputs to the model are:
     D  Angle  V_in                                       outputs_file
0  200     40   0.5  /gpfs/fs2/scratch/mgholiza/test_installable_al...
Model inputs are saved in:
 /gpfs/fs2/scratch/mgholiza/test_installable_alcfd/alcfd/examples/inputs_par_test.txt
Results are stored in:
 /gpfs/fs2/scratch/mgholiza/test_installable_alcfd/alcfd/examples/outputs_par_test.txt.


,Inlet D (um),Expansion angle,Mesh size,Viscosity,Density,Inlet P,Inlet V,System vol,Backflow percent,Backflow vol
0,200.0,40.0,0.000025,0.004,1060.0,15998.7,0.5,3.528010e-10,48.775337,1.720799e-10
